In [1]:
from typing import List, Union

from dotenv import load_dotenv
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.tools import Tool
from langchain.tools.render import render_text_description

from callbacks import AgentCallbackHandler

AgentCallbackHandler

@tool
def get_tex_length(text: str) -> int:
    """Returns the length of a text by characters"""
    print(f"get_text_lenght enter with {text=} ")
    text = text.strip("'\n").strip('"')
    return len(text)


def find_tool_by_name(tools: List[Tool], tool_name: str):
    for tool in tools:
        if tool.name == tool_name:
            return tool
    raise ValueError(f"Tool name:{tool_name} not found")


if __name__ == "__main__":
    print("Hello ReACT Langchain")
    tools = [get_tex_length]

    # Podemos chamar esse prompt de cadeia de pensamento, porque pedimos ao llm que nos diga como está pensando e chegando a sua resposta
    template = """
    Answer the following questions as best you can. You have access to the following tools:

    {tools}

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
        
    """

    prompt = PromptTemplate.from_template(template=template).partial(
        tools=render_text_description(tools),
        tool_names=", ".join([t.name for t in tools]),
    )

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, stop=["\nObservation"], callbacks=[AgentCallbackHandler()])
    intermediate_steps = []
    
    
    from operator import itemgetter


    
    agent = (
        {
            "input": RunnablePassthrough(),
            'agent_scratchpad': lambda x: format_log_to_str(x['agent_scratchpad']),
        }
        | prompt 
        | llm 
        | ReActSingleInputOutputParser()
    )

    agent_step: Union[AgentAction, AgentFinish] = agent.invoke(
        {
         "input": 'What is the length of "DOG" in characters',
         'agent_scratchpad':intermediate_steps
        }
    )
    
    print(agent_step)

    if isinstance(agent_step, AgentAction):
        tool_name = agent_step.tool
        tool_to_use = find_tool_by_name(tools,tool_name) 
        tool_input = agent_step.tool_input
        
        observation = tool_to_use.func(str(tool_input))
        print(f'{observation=}')   
        intermediate_steps.append((agent_step, str(observation)))
        
    agent_step: Union[AgentAction, AgentFinish] = agent.invoke(
        {
         "input": 'What is the length of "DOG" in characters',
         'agent_scratchpad':intermediate_steps
        }
    )
    
    if isinstance(agent_step, AgentFinish):
        print(agent_step.return_values)
    

Hello ReACT Langchain
***Prompt to LLM 
Human: 
    Answer the following questions as best you can. You have access to the following tools:

    get_tex_length(text: str) -> int - Returns the length of a text by characters

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [get_tex_length]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: {'input': 'What is the length of "DOG" in characters', 'agent_scratchpad': []}
    Thought: 
        
    
***
*** LLM response:*** 
I need to determine the length of the string "DOG" in characters. 

Action: get_tex_length  
Action Input: "DOG"  
****
tool='get_tex_length' tool_inpu